<a href="https://colab.research.google.com/github/IRPARKS/NMML/blob/main/NMMLHW13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Vichoko/pytorch-wavenet.git

Cloning into 'pytorch-wavenet'...
remote: Enumerating objects: 1168, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1168 (delta 3), reused 6 (delta 3), pack-reused 1158
Receiving objects: 100% (1168/1168), 268.95 MiB | 20.86 MiB/s, done.
Resolving deltas: 100% (720/720), done.


In [2]:
import sys
sys.path.append("pytorch-wavenet")

import torch
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *

In [3]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

use gpu


In [4]:
model = WaveNetModel(layers=3,
                     blocks=2,
                     dilation_channels=8,
                     residual_channels=8,
                     skip_channels=64,
                     end_channels=32,
                     output_length=4,
                     dtype=dtype,
                     bias=True)
# model = load_latest_model_from('snapshots', use_cuda=use_cuda)

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

model:  WaveNetModel(
  (filter_convs): ModuleList(
    (0-5): 6 x Conv1d(8, 8, kernel_size=(2,), stride=(1,))
  )
  (gate_convs): ModuleList(
    (0-5): 6 x Conv1d(8, 8, kernel_size=(2,), stride=(1,))
  )
  (residual_convs): ModuleList(
    (0-5): 6 x Conv1d(8, 8, kernel_size=(1,), stride=(1,))
  )
  (skip_convs): ModuleList(
    (0-5): 6 x Conv1d(8, 64, kernel_size=(1,), stride=(1,))
  )
  (start_conv): Conv1d(256, 8, kernel_size=(1,), stride=(1,))
  (end_conv_1): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
  (end_conv_2): Conv1d(32, 256, kernel_size=(1,), stride=(1,))
)
receptive field:  15
parameter count:  18104


/content/pytorch-wavenet/wavenet_modules.py:53: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  self.data = Variable(dtype(num_channels, max_length).zero_())


In [5]:
data = WavenetDataset(dataset_file='/content/pytorch-wavenet/train_samples/bach_chaconne/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='train_samples/bach_chaconne',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')

one hot input
the dataset has 2393867 items


In [6]:
def generate_and_log_samples(step):
    sample_length=32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False)
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")

logger = Logger(log_interval=200,
                 validation_interval=400,
                 generate_interval=1000)

In [7]:
trainer = WavenetTrainer(model=model,
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='chaconne_model',
                         snapshot_interval=1000,
                         logger=logger,
                         dtype=dtype,
                         ltype=ltype)

print('start training...')
trainer.train(batch_size=16, epochs=10)

# Function to generate and log audio samples
def generate_and_log_samples(step):
    sample_length = 32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False)
    print("start generating...")

    # Generate audio samples with temperature 0.5
    samples_05 = generate_audio(gen_model, length=sample_length, temperatures=[0.5])
    tf_samples_05 = tf.convert_to_tensor(samples_05, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples_05, step, sr=16000)

    # Generate audio samples with temperature 1.0
    samples_10 = generate_audio(gen_model, length=sample_length, temperatures=[1.0])
    tf_samples_10 = tf.convert_to_tensor(samples_10, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples_10, step, sr=16000)

    print("audio clips generated")

# Call generate_and_log_samples at specified intervals during training
for step in range(0, 10001, 1000):  # Example: Generate every 1000 steps
    generate_and_log_samples(step)

start training...
epoch 0


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
start_data = data[250000][0] # use start data from the data set
start_data = torch.max(start_data, 0)[1] # convert one hot vectors to integers

def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")

generated = model.generate_fast(num_samples=160000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)